In [1]:
import numpy as np

In [2]:
from trsm import TRSM, combos_6j

In [3]:
import vector

In [4]:
filename = f'../../NMSSM_XYH_YToHH_6b_MX_700_MY_400_6jet_testing_set_7jets_2021Aug.root'
trsm = TRSM(filename=filename)

In [5]:
combos7 = combos_6j(trsm, 7)

100%|██████████████████████████████████████████████████████| 129355/129355 [03:57<00:00, 544.71it/s]
ic| combo_p4.pt: <Array [[120, 115, 76, ... 41.7, 40, 35.9]] type='905485 * var * float64'>
ic| type(combo_p4.pt): <class 'awkward.highlevel.Array'>


Total events chosen: 129355


In [6]:
pt_7  = combos7.combo_features[:,0:6]
eta_7 = combos7.combo_features[:,6:12]
phi_7 = combos7.combo_features[:,12:18]
m_7   = np.ones(pt_7.shape)*4

In [7]:
p4_7_0 = vector.obj(pt=pt_7[:,0], eta=eta_7[:,0], phi=phi_7[:,0], mass=m_7[:,0])
p4_7_1 = vector.obj(pt=pt_7[:,1], eta=eta_7[:,1], phi=phi_7[:,1], mass=m_7[:,1])
p4_7_2 = vector.obj(pt=pt_7[:,2], eta=eta_7[:,2], phi=phi_7[:,2], mass=m_7[:,2])
p4_7_3 = vector.obj(pt=pt_7[:,3], eta=eta_7[:,3], phi=phi_7[:,3], mass=m_7[:,3])
p4_7_4 = vector.obj(pt=pt_7[:,4], eta=eta_7[:,4], phi=phi_7[:,4], mass=m_7[:,4])
p4_7_5 = vector.obj(pt=pt_7[:,5], eta=eta_7[:,5], phi=phi_7[:,5], mass=m_7[:,5])

In [8]:
p4_7 = p4_7_0 + p4_7_1 + p4_7_2 + p4_7_3 + p4_7_4 + p4_7_5

In [9]:
p4_7 = combos7.sixjet_p4

In [10]:
import awkward as ak

In [11]:
sgnl_mass_7 = ak.to_numpy(p4_7.mass[combos7.sgnl_mask])
bkgd_mass_7 = ak.to_numpy(p4_7.mass[~combos7.sgnl_mask])

In [12]:
from lmfit.models import GaussianModel

bins = np.linspace(300,1100,150)
n_s, b_s = np.histogram(sgnl_mass_7, bins=bins)

x = (b_s[1:] + b_s[:-1]) / 2
y = n_s

mod = GaussianModel()

pars = mod.guess(y, x=x)
out = mod.fit(y, pars, x=x)

print(out.fit_report(min_correl=0.25))

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 21
    # data points      = 149
    # variables        = 3
    chi-square         = 151070.705
    reduced chi-square = 1034.73086
    Akaike info crit   = 1037.31198
    Bayesian info crit = 1046.32382
[[Variables]]
    amplitude:  237165.215 +/- 1242.09946 (0.52%) (init = 333455)
    center:     669.022491 +/- 0.31583334 (0.05%) (init = 667.7852)
    sigma:      52.2253080 +/- 0.31583334 (0.60%) (init = 59.0604)
    fwhm:       122.981200 +/- 0.74373067 (0.60%) == '2.3548200*sigma'
    height:     1811.67407 +/- 9.48826756 (0.52%) == '0.3989423*amplitude/max(1e-15, sigma)'
[[Correlations]] (unreported correlations are < 0.250)
    C(amplitude, sigma) =  0.577


In [13]:
out.best_values

{'amplitude': 237165.2145737275,
 'center': 669.0224912511314,
 'sigma': 52.225308006606774}

In [14]:
A = out.best_values['amplitude']
c = out.best_values['center']
s = out.best_values['sigma']

In [15]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [16]:
min_mass = c - 2*s
max_mass = c + 2*s

In [17]:
min_mass

564.5718752379179

In [18]:
max_mass

773.473107264345

In [19]:
sgnl_mass_mask = np.logical_and(sgnl_mass_7 > min_mass, sgnl_mass_7 < max_mass)
bkgd_mass_mask = np.logical_and(bkgd_mass_7 > min_mass, bkgd_mass_7 < max_mass)

In [20]:
len(bkgd_mass_7[bkgd_mass_mask])/len(bkgd_mass_7)

0.46174401546782134

In [21]:
%matplotlib widget
import matplotlib.pyplot as plt

In [22]:
from consistent_plots import hist

In [23]:
fig, ax = plt.subplots()
ax.set_title(r"$^7C_6$ Combinations Normalized")

bins = np.linspace(300,1100,150)

n_s, b_s = np.histogram(sgnl_mass_7[sgnl_mass_mask], bins=bins)
n_b, b_b = np.histogram(bkgd_mass_7[bkgd_mass_mask], bins=bins)
n, b = np.histogram(ak.to_numpy(p4_7.mass), bins=bins)

x = (b_s[1:] + b_s[:-1])/2

gaus = gaussian(x, c, s)
_ = hist(ax, x, weights=n_s/n.max(), bins=bins, label='Signal')
_ = hist(ax, x, weights=n_b/n.max(), bins=bins, label='Background')
_ = ax.plot(x, n_s.max()*gaus/n.max())

# _ = hist(ax, x, weights=n/n.max(), label='All')
ax.legend(loc=1, fontsize='small')
ax.set_xlabel(r'Invariant Mass [GeV]')
ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [24]:
len(sgnl_mass_mask) / (len(sgnl_mass_mask) + len(bkgd_mass_mask))

0.05068443983058803